# トレーニングスクリプトをコマンドジョブとして実行する

Python SDK for Azure Machine Learningを使用すると、スクリプトをコマンドジョブとして投入できます。ジョブを利用することで、機械学習モデルを学習する際の入力パラメーターや出力を簡単に把握することができます。

## 始める前に

このノートブックのコードを実行するには、最新バージョンの **azure-ai-ml** パッケージが必要です。以下のセルを実行して、インストールされていることを確認してください。

> 注**：
> もし **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールしてください。

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.21.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## ワークスペースに接続する

必要な SDK パッケージがインストールされ、ワークスペースに接続する準備が整いました。

ワークスペースに接続するには、サブスクリプションID、リソースグループ名、ワークスペース名といった識別子のパラメータが必要だ。Azure Machine Learningが管理するコンピュートインスタンスで作業しているので、デフォルト値を使用してワークスペースに接続できる。

In [2]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## MLflow によるカスタムトラッキング

スクリプトをジョブとして実行する場合、トレーニングスクリプトで MLflow を使用してモデルを追跡することができます。MLflow では、ジョブの出力と一緒に保存したいカスタムパラメータ、メトリクス、成果物を追跡することができます。

以下のセルを実行して、**src** フォルダに **train-model-mlflow.py** スクリプトを作成します。このスクリプトは、引数として渡される同じフォルダ内の **diabetes.csv** ファイルを使用して分類モデルをトレーニングします。

スクリプトが `mlflow` とログをインポートすることを見つけるために、以下のコードを見てください：

- パラメータ**として正則化率。
- パラメータ**としての正則化率。
- プロットされた ROC 曲線。

In [4]:
import os

# create a folder for the script files
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

src folder created


In [5]:
%%writefile $script_folder/train-model-mlflow.py
# import libraries
import mlflow
import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

def main(args):
    # read data
    df = get_data(args.training_data)

    # split data
    X_train, X_test, y_train, y_test = split_data(df)

    # train model
    model = train_model(args.reg_rate, X_train, X_test, y_train, y_test)

    # evaluate model
    eval_model(model, X_test, y_test)

# function that reads the data
def get_data(path):
    print("Reading data...")
    df = pd.read_csv(path)
    
    return df

# function that splits the data
def split_data(df):
    print("Splitting data...")
    X, y = df[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness',
    'SerumInsulin','BMI','DiabetesPedigree','Age']].values, df['Diabetic'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

    return X_train, X_test, y_train, y_test

# function that trains the model
def train_model(reg_rate, X_train, X_test, y_train, y_test):
    mlflow.log_param("Regularization rate", reg_rate)
    print("Training model...")
    model = LogisticRegression(C=1/reg_rate, solver="liblinear").fit(X_train, y_train)

    return model

# function that evaluates the model
def eval_model(model, X_test, y_test):
    # calculate accuracy
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)
    print('Accuracy:', acc)
    mlflow.log_metric("Accuracy", acc)

    # calculate AUC
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test,y_scores[:,1])
    print('AUC: ' + str(auc))
    mlflow.log_metric("AUC", auc)

    # plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
    fig = plt.figure(figsize=(6, 4))
    # Plot the diagonal 50% line
    plt.plot([0, 1], [0, 1], 'k--')
    # Plot the FPR and TPR achieved by our model
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.savefig("ROC-Curve.png")
    mlflow.log_artifact("ROC-Curve.png")    

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", dest='training_data',
                        type=str)
    parser.add_argument("--reg_rate", dest='reg_rate',
                        type=float, default=0.01)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Writing src/train-model-mlflow.py


これで、スクリプトをコマンド・ジョブとして投入できる。
以下のセルを実行してモデルをトレーニングします。

In [6]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-mlflow.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-mlflow",
    experiment_name="diabetes-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.53 MBs): 100%|███████

Monitor your job at https://ml.azure.com/runs/musing_sock_rgc0ct58r1?wsid=/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo&tid=adcb56ab-ed63-4be3-8e5e-2fa7d2eb44e1


スタジオで、**diabetes-train-mlflow** ジョブに移動し、実行したコマンドジョブの概要を調べます：
- Overview**]タブの[**Params**]で、ログに記録されたパラメータを検索します。
- ログに記録されたメトリクスを **Metrics** タブで見つけます。
- Images**タブ、および**Outputs + logs**タブ（すべてのファイル）で、ログに記録されたアーティファクトを見つけます。

## MLflow による自動ロギング
カスタムロギングを使用する代わりに、MLflow は任意のパラメータ、メトリクス、および成果物を自動的にロギングすることもできます。MLflowによるオートロギングに必要なコードは1行だけです。

以下のセルを実行して、**src** フォルダに **train-model-autolog.py** スクリプトを作成します。このスクリプトは、引数として渡される同じフォルダの **diabetes.csv** ファイルを用いて分類モデルを学習します。

以下のコードを見て、スクリプトが `mlflow` をインポートし、次の行でオートログを有効にしていることを確認する： 

mlflow.autolog()`

In [7]:
%%writefile $script_folder/train-model-autolog.py
# import libraries
import mlflow
import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

def main(args):
    # enable autologging
    mlflow.autolog()

    # read data
    df = get_data(args.training_data)

    # split data
    X_train, X_test, y_train, y_test = split_data(df)

    # train model
    model = train_model(args.reg_rate, X_train, X_test, y_train, y_test)

    eval_model(model, X_test, y_test)

# function that reads the data
def get_data(path):
    print("Reading data...")
    df = pd.read_csv(path)
    
    return df

# function that splits the data
def split_data(df):
    print("Splitting data...")
    X, y = df[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness',
    'SerumInsulin','BMI','DiabetesPedigree','Age']].values, df['Diabetic'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

    return X_train, X_test, y_train, y_test

# function that trains the model
def train_model(reg_rate, X_train, X_test, y_train, y_test):
    print("Training model...")
    model = LogisticRegression(C=1/reg_rate, solver="liblinear").fit(X_train, y_train)

    return model

# function that evaluates the model
def eval_model(model, X_test, y_test):
    # calculate accuracy
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)
    print('Accuracy:', acc)

    # calculate AUC
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test,y_scores[:,1])
    print('AUC: ' + str(auc))

    # plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
    fig = plt.figure(figsize=(6, 4))
    # Plot the diagonal 50% line
    plt.plot([0, 1], [0, 1], 'k--')
    # Plot the FPR and TPR achieved by our model
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.savefig("ROC-Curve.png") 

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", dest='training_data',
                        type=str)
    parser.add_argument("--reg_rate", dest='reg_rate',
                        type=float, default=0.01)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Writing src/train-model-autolog.py


これで、スクリプトをコマンド・ジョブとして投入できる。
以下のセルを実行してモデルをトレーニングします。

In [8]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-autolog.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-autolog",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Uploading src (0.53 MBs): 100%|██████████| 531069/531069 [00:00<00:00, 10049695.82it/s]




Monitor your job at https://ml.azure.com/runs/jolly_napa_bhq90pxcn8?wsid=/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo&tid=adcb56ab-ed63-4be3-8e5e-2fa7d2eb44e1


スタジオで、**diabetes-train-autolog** ジョブに移動し、実行したコマンドジョブの概要を調べます：
- Overview**]タブの[**Params**]で、ログに記録されたパラメータを見つけます。
- ログに記録されたメトリクスを **Metrics** タブで見つけます。
- Images**タブ（特に画像）、および**Outputs + logs**タブ（モデルファイルを含むすべてのファイル）で、ログに記録されたアーチファクトを見つけます。

## MLflow を使って実験の表示と検索を行う
Azure Machine Learning Studio は、ジョブの実行を表示および比較するための使いやすい UI です。また、MLflow を使用して実験ジョブを表示することもできます。

ワークスペース内の実験を一覧表示するには、以下のコマンドを使用します：

In [9]:
import mlflow
experiments = mlflow.search_experiments()
for exp in experiments:
    print(exp.name)

Default
demoproject
move-diabetes-data
diabetes-training
prepare_image
auto-ml-class-dev
mlflow-experiment-diabetes


特定の実験を検索するには、その実験名で検索することができる：

In [10]:
experiment_name = "diabetes-training"
exp = mlflow.get_experiment_by_name(experiment_name)
print(exp)

<Experiment: artifact_location='', creation_time=1730418618805, experiment_id='b2f1aba9-55fe-4749-99aa-e95d458fc300', last_update_time=None, lifecycle_stage='active', name='diabetes-training', tags={}>


実験名を指定すると、その実験のすべてのジョブを検索できます：

In [11]:
mlflow.search_runs(exp.experiment_id)

run_id                         experiment_id     status  \
0    lucid_door_dswxxpm5ks  b2f1aba9-55fe-4749-99aa-e95d458fc300   FINISHED   
1    lime_zebra_v4hrbnz2m3  b2f1aba9-55fe-4749-99aa-e95d458fc300   FINISHED   
2   sad_curtain_tdt1gthn51  b2f1aba9-55fe-4749-99aa-e95d458fc300     FAILED   
3  sweet_parcel_2n9fgq4p54  b2f1aba9-55fe-4749-99aa-e95d458fc300   FINISHED   
4  amiable_leek_lxb0ljsyrw  b2f1aba9-55fe-4749-99aa-e95d458fc300    RUNNING   
5   musing_sock_rgc0ct58r1  b2f1aba9-55fe-4749-99aa-e95d458fc300  SCHEDULED   
6    jolly_napa_bhq90pxcn8  b2f1aba9-55fe-4749-99aa-e95d458fc300  SCHEDULED   

  artifact_uri                       start_time  \
0              2024-10-31 23:53:13.986000+00:00   
1              2024-11-01 00:05:22.558000+00:00   
2              2024-11-01 00:06:53.963000+00:00   
3              2024-11-01 00:27:58.410000+00:00   
4              2024-11-12 01:23:25.900000+00:00   
5                     1970-01-01 00:00:00+00:00   
6                     1970-01-01 00:00:00+00:00   

                          end_time    tags.mlflow.rootRunId tags.mlflow.user  \
0 2024-10-31 23:54:50.941000+00:00    lucid_door_dswxxpm5ks             歩 小池   
1 2024-11-01 00:06:56.513000+00:00    lime_zebra_v4hrbnz2m3             歩 小池   
2 2024-11-01 00:07:45.676000+00:00   sad_curtain_tdt1gthn51             歩 小池   
3 2024-11-01 00:29:21.088000+00:00  sweet_parcel_2n9fgq4p54             歩 小池   
4                              NaT  amiable_leek_lxb0ljsyrw             歩 小池   
5                              NaT   musing_sock_rgc0ct58r1             歩 小池   
6                              NaT    jolly_napa_bhq90pxcn8             歩 小池   

          tags.mlflow.runName     tags.model_type  
0      diabetes-train-cluster                None  
1  diabetes-train-curated-env                None  
2   diabetes-train-custom-env                None  
3   diabetes-train-custom-env                None  
4       diabetes-train-script                None  
5       diabetes-train-mlflow  LogisticRegression  
6      diabetes-train-autolog                None

ジョブの実行と出力をより簡単に比較するために、検索結果を順番に並べるように設定することができます。例えば、次のセルは `start_time` によって結果を並べ替え、最大で `2` の結果のみを表示する： 

In [12]:
mlflow.search_runs(exp.experiment_id, order_by=["start_time DESC"], max_results=2)

run_id                         experiment_id    status  \
0  amiable_leek_lxb0ljsyrw  b2f1aba9-55fe-4749-99aa-e95d458fc300   RUNNING   
1  sweet_parcel_2n9fgq4p54  b2f1aba9-55fe-4749-99aa-e95d458fc300  FINISHED   

  artifact_uri                       start_time  \
0              2024-11-12 01:23:25.900000+00:00   
1              2024-11-01 00:27:58.410000+00:00   

                          end_time    tags.mlflow.rootRunId tags.mlflow.user  \
0                              NaT  amiable_leek_lxb0ljsyrw             歩 小池   
1 2024-11-01 00:29:21.088000+00:00  sweet_parcel_2n9fgq4p54             歩 小池   

         tags.mlflow.runName  
0      diabetes-train-script  
1  diabetes-train-custom-env

クエリを作成して、実行をフィルタリングすることもできる。フィルタリングのクエリー文字列はSQLの`WHERE`句の簡略版で記述されます。

フィルタリングには、2種類のコンパレータを使用することができます：

- 数値比較子（メトリクス）： 数値比較子（メトリクス）： =, !=, >, >=, <, <=。
- 文字列比較子（パラメータ、タグ、属性）：=と!

MLflowで実験を追跡する方法](https://learn.microsoft.com/azure/machine-learning/how-to-track-experiments-mlflow)の詳細はこちら。

In [13]:
query = "metrics.AUC > 0.8 and tags.model_type = 'LogisticRegression'"
mlflow.search_runs(exp.experiment_id, filter_string=query)

Empty DataFrame
Columns: [run_id, experiment_id, status, artifact_uri, start_time, end_time]
Index: []